In [1]:
import sys
import os
import torch
import numpy as np
import torchvision
import cv2
import WDNet

basic_path = ''
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
torch.cuda.is_available()

True

In [2]:
torch.cuda.empty_cache()
args ={}
args['epoch'] = 1
args['batch_size'] = 24 # 12 for 16Gb
args['save_dir'] = basic_path+'../model_WDNet'
args['result_dir'] = ''
args['dataset'] = basic_path+'../data_CLWD'
args['log_dir'] = basic_path+'../model_WDNet'
args['gpu_mode'] = True
args['input_size'] = 28 # не знаю что это
args['dataloader_workers'] = 8
args['gan_type'] = 'CGAN'
args['lrG'] = 0.0002
args['lrD'] = 0.0002
args['beta1'] = 0.5
args['beta2'] = 0.999
args


{'epoch': 1,
 'batch_size': 24,
 'save_dir': '../model_WDNet',
 'result_dir': '',
 'dataset': '../data_CLWD',
 'log_dir': '../model_WDNet',
 'gpu_mode': True,
 'input_size': 28,
 'dataloader_workers': 8,
 'gan_type': 'CGAN',
 'lrG': 0.0002,
 'lrD': 0.0002,
 'beta1': 0.5,
 'beta2': 0.999}

In [3]:
wdnet = WDNet.WDNet(args)

Model loaded from ../model_WDNet


In [4]:
wdnet.train()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /tmp/xdg_cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 68.9MB/s] 


Start training!
Epoch: [ 1] [ 100/2500] D_loss: 0.67252433, G_loss: 0.83240670
Epoch: [ 1] [ 200/2500] D_loss: 0.67297792, G_loss: 0.96195859
Epoch: [ 1] [ 300/2500] D_loss: 0.66326904, G_loss: 0.84323102
Epoch: [ 1] [ 400/2500] D_loss: 0.70419419, G_loss: 0.83807433
Epoch: [ 1] [ 500/2500] D_loss: 0.64107430, G_loss: 0.80234998
Epoch: [ 1] [ 600/2500] D_loss: 0.65512776, G_loss: 0.72187632
Epoch: [ 1] [ 700/2500] D_loss: 0.65598786, G_loss: 0.97517383
Epoch: [ 1] [ 800/2500] D_loss: 0.64846826, G_loss: 0.70796013
Epoch: [ 1] [ 900/2500] D_loss: 0.73134184, G_loss: 0.80687582
Epoch: [ 1] [1000/2500] D_loss: 0.63538778, G_loss: 0.73424172
Epoch: [ 1] [1100/2500] D_loss: 0.65784067, G_loss: 0.67114413
Epoch: [ 1] [1200/2500] D_loss: 0.68963623, G_loss: 0.82628238
Epoch: [ 1] [1300/2500] D_loss: 0.66105801, G_loss: 0.69155097
Epoch: [ 1] [1400/2500] D_loss: 0.65002227, G_loss: 0.90541130
Epoch: [ 1] [1500/2500] D_loss: 0.62928027, G_loss: 0.80564499
Epoch: [ 1] [1600/2500] D_loss: 0.61204